# Converting TensorFlow Model

## Using Frozen Graph

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import array
import cv2

In [ ]:
ds, ds_info = tfds.load('imagenet_v2', split='test', as_supervised=True, with_info=True)
ds = tfds.as_numpy(ds)

test_labels = []
img_list = []

In [ ]:
NUM_TEST_SAMPLES = 100
IMAGE_SIDE = 96

def dumpLayers(model):
    for layer in model.layers:
        print(layer.name)
        break

def getModel():
    model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3), alpha=0.35,
        include_top=True, classifier_activation='softmax', weights='imagenet', pooling='avg'
    )
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    return model

def preprocess(image):
    image = (image) / 127.5
    return image - 1.0

def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

def quantizationDataGenerator():
    count = 0
    for image, label in ds:
        image = image.copy()
        image = cv2.resize(image, (IMAGE_SIDE, IMAGE_SIDE))
        array = preprocess(image).astype(np.float32)[np.newaxis, ...]
        yield([array])
        count += 1
        if count > 300:
            break

def testDataGenerator():
    count = 0
    for image, label in ds:
        image = image.copy()
        image = cv2.resize(image, (IMAGE_SIDE, IMAGE_SIDE))
        imshow(image)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite('img/' + str(label) + '.jpg', img_rgb, [cv2.IMWRITE_JPEG_QUALITY,100])
        print(img_rgb)
        array = preprocess(image).astype(np.float32)[np.newaxis, ...]
        print(label)
        test_labels.append(label)
        yield(array)
        count += 1
        if count > 300:
            break
        
def convertTFL(name, model):
    converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph('generated/mobilenet/mobilenet_v2_0.35_96_frozen.pb', input_arrays = ['input'], output_arrays = ['MobilenetV2/Predictions/Reshape'], input_shapes = {'input' : [1, 96, 96, 3]})
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = quantizationDataGenerator
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    quantModel = converter.convert()

    with open("{}.tflite".format(name), "wb") as f:
        f.write(quantModel)
        
def runTFL(name):
    from tflite_runtime.interpreter import Interpreter
    interpreter = Interpreter('{}.tflite'.format(name))
    interpreter.allocate_tensors()

    inputTensorIndex = interpreter.get_input_details()[0]['index']
    outputTensorIndex = interpreter.get_output_details()[0]['index']
    input_scale, input_zero_point = interpreter.get_input_details()[0]['quantization']
    output_scale, output_zero_point = interpreter.get_output_details()[0]['quantization']
    
    print(input_scale)
    print(input_zero_point)
    
    prediction_values = []
    
    for x in testDataGenerator():
        test_image = np.int8(x / input_scale + input_zero_point)
        interpreter.set_tensor(inputTensorIndex, test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(outputTensorIndex)
        result = np.argmax(output[0])
        prediction_values.append(result - 1)
    
    accurate_count = 0
    for index in range(len(prediction_values)):
        print(str(prediction_values[index]) + ":" + str(test_labels[index]))
        if prediction_values[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_values)

    return accuracy * 100

In [ ]:
tf.keras.backend.clear_session()

name = 'mobilenet_v2'
model = getModel()
model.summary()
convertTFL(name, model)

In [ ]:
runTFL(name)

In [ ]:
file = tf.keras.utils.get_file(
    "mountains.jpg",
    "https://storage.googleapis.com/gcptutorials.com/examples/mountains.jpg")
img = tf.keras.preprocessing.image.load_img(file, target_size=[96, 96])

x = tf.keras.preprocessing.image.img_to_array(img)
print(x.shape)
imshow(x)
x = tf.keras.applications.mobilenet.preprocess_input(x[tf.newaxis,...])
print(x.shape)

labels_path = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
labels = np.array(open(labels_path).read().splitlines())
print(len(labels))

predictions = model(x)
print(predictions.shape)

top_5_classes_index = np.argsort(predictions)[0 , ::-1][:5]+1

print(top_5_classes_index)

top_5_classes = labels[top_5_classes_index]
print(top_5_classes)



from tflite_runtime.interpreter import Interpreter
interpreter = Interpreter('mobilenet_v2.tflite'.format(name))
interpreter.allocate_tensors()

inputTensorIndex = interpreter.get_input_details()[0]['index']
outputTensorIndex = interpreter.get_output_details()[0]['index']
scale, zero_point = interpreter.get_input_details()[0]['quantization']

print(scale)
print(zero_point)

prediction_values = []

test_image = np.int8(x / scale + zero_point)
interpreter.set_tensor(inputTensorIndex, test_image)
interpreter.invoke()
output = interpreter.get_tensor(outputTensorIndex)
result = np.argmax(output[0])
print (result)

## Done